## Torch

In [1]:
import torch
torch.__version__

In [2]:
torch_model = torch.load("./pytorch_model.bin",map_location='cpu')# luke github官方上面提供的squad微调后权重
torch_model

OrderedDict([('encoder.layer.0.attention.self.query.weight',
              tensor([[-0.0034,  0.0348,  0.0006,  ...,  0.0017,  0.0590, -0.0426],
                      [-0.0249,  0.0531, -0.0148,  ..., -0.0304, -0.0145,  0.0117],
                      [ 0.0062,  0.0708, -0.0332,  ...,  0.0803,  0.0118, -0.0132],
                      ...,
                      [-0.0585,  0.0210, -0.0427,  ..., -0.0298,  0.0051,  0.0694],
                      [ 0.0424,  0.0232, -0.0613,  ..., -0.0552, -0.0157,  0.0171],
                      [-0.0183, -0.0454, -0.0102,  ...,  0.0471,  0.0229, -0.0177]])),
             ('encoder.layer.0.attention.self.query.bias',
              tensor([ 0.3127,  0.0561, -0.0748,  ..., -0.0707, -0.0500, -0.0668])),
             ('encoder.layer.0.attention.self.key.weight',
              tensor([[-0.0044, -0.0183, -0.0138,  ..., -0.0041,  0.0094, -0.0148],
                      [-0.0240, -0.0002,  0.0252,  ...,  0.0400,  0.0435, -0.0201],
                      [-0.0266, -0

In [3]:
torch_model['embeddings.word_embeddings.weight'].shape

torch.Size([50265, 1024])

In [23]:
 for weight_name, weight_value in torch_model.items():
        print(weight_value.shape)
        y=weight_value.numpy()
        print(y)

torch.Size([1024, 1024])
[[-0.00344849  0.03479004  0.00061941 ...  0.00167656  0.059021
  -0.04260254]
 [-0.02487183  0.05307007 -0.01480865 ... -0.03038025 -0.01448059
   0.01166534]
 [ 0.00621414  0.07080078 -0.03323364 ...  0.08026123  0.01176453
  -0.01317596]
 ...
 [-0.05853271  0.02101135 -0.04269409 ... -0.02983093  0.00513077
   0.06939697]
 [ 0.04238892  0.02316284 -0.0612793  ... -0.05517578 -0.01571655
   0.01708984]
 [-0.01829529 -0.04544067 -0.01016998 ...  0.04708862  0.02294922
  -0.01771545]]
torch.Size([1024])
[ 0.31274414  0.05606079 -0.0748291  ... -0.07067871 -0.04995728
 -0.06677246]
torch.Size([1024, 1024])
[[-0.00439072 -0.01828003 -0.01376343 ... -0.0040741   0.00939941
  -0.01475525]
 [-0.02403259 -0.0002172   0.02523804 ...  0.04003906  0.04351807
  -0.02008057]
 [-0.02658081 -0.05276489 -0.01207733 ... -0.03643799  0.00733566
   0.0145874 ]
 ...
 [-0.07049561 -0.02668762 -0.01916504 ... -0.01908875  0.00925446
   0.10125732]
 [ 0.01512146  0.00782013 -0.0169

## mindspore

In [5]:
from readingcomprehension.models.luke import LukeForReadingComprehension
import mindspore.common.dtype as mstype
from model.bert_model import BertConfig
from mindspore import context
from model.luke import LukeModel, EntityAwareEncoder
import numpy as np
from mindspore import Tensor, context
from mindspore import dtype as mstype
import mindspore.ops as ops
import mindspore.nn as nn
from model.bert_model import BertOutput
from mindspore.common.initializer import TruncatedNormal
import math
from mindspore.ops import composite as C
import mindspore
from mindspore.ops import operations as P
context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

In [6]:
config = BertConfig()
model = LukeForReadingComprehension(config)
ms_model = model.parameters_dict()
ms_model

OrderedDict([('encoder.layer.0.attention.self.query.weight',
              Parameter (name=encoder.layer.0.attention.self.query.weight, shape=(1024, 1024), dtype=Float32, requires_grad=True)),
             ('encoder.layer.0.attention.self.query.bias',
              Parameter (name=encoder.layer.0.attention.self.query.bias, shape=(1024,), dtype=Float32, requires_grad=True)),
             ('encoder.layer.0.attention.self.key.weight',
              Parameter (name=encoder.layer.0.attention.self.key.weight, shape=(1024, 1024), dtype=Float32, requires_grad=True)),
             ('encoder.layer.0.attention.self.key.bias',
              Parameter (name=encoder.layer.0.attention.self.key.bias, shape=(1024,), dtype=Float32, requires_grad=True)),
             ('encoder.layer.0.attention.self.value.weight',
              Parameter (name=encoder.layer.0.attention.self.value.weight, shape=(1024, 1024), dtype=Float32, requires_grad=True)),
             ('encoder.layer.0.attention.self.value.bias',
  

In [7]:
ms_model['embeddings.word_embeddings.embedding_table']

Parameter (name=embeddings.word_embeddings.embedding_table, shape=(50267, 1024), dtype=Float32, requires_grad=True)

## torch2ms

In [27]:
import os
import collections
from mindspore import log as logger
from mindspore.common.tensor import Tensor
from mindspore.common.initializer import initializer
from mindspore import save_checkpoint
from mindspore import Parameter

In [17]:
def build_params_map(attention_num=16):
    """
    build params map from torch's LUKE to mindspore's LUKE
    map=> key：value，torch_name：ms_name
    键：torch权重名称，值：mindspore权重名称
    :return:
    """
    weight_map = collections.OrderedDict({
        'embeddings.word_embeddings.weight': "embeddings.word_embeddings.embedding_table",
        'embeddings.position_embeddings.weight': "embeddings.position_embeddings.embedding_table",
        'embeddings.token_type_embeddings.weight': "embeddings.token_type_embeddings.embedding_table",
        'embeddings.LayerNorm.weight': 'embeddings.LayerNorm.gamma',
        'embeddings.LayerNorm.bias': 'embeddings.LayerNorm.beta',
        'entity_embeddings.entity_embeddings.weight':'entity_embeddings.entity_embeddings.embedding_table',
        'entity_embeddings.entity_embedding_dense.weight':'entity_embeddings.entity_embedding_dense.weight',
        'entity_embeddings.position_embeddings.weight':'entity_embeddings.position_embeddings.embedding_table',
        'entity_embeddings.token_type_embeddings.weight':'entity_embeddings.token_type_embeddings.embedding_table',
        'entity_embeddings.LayerNorm.weight':'entity_embeddings.layer_norm.gamma',
        'entity_embeddings.LayerNorm.bias':'entity_embeddings.layer_norm.beta',
        'qa_outputs.weight':'qa_outputs.weight',
        'qa_outputs.bias':'qa_outputs.bias',
        'pooler.dense.weight':'pooler.weight',
        'pooler.dense.bias':'pooler.bias'
        
    })
    
    # add attention layers
    for i in range(attention_num):
        weight_map[f'encoder.layer.{i}.attention.self.query.weight'] = \
            f'encoder.layer.{i}.attention.self_attention.query.weight'
        weight_map[f'encoder.layer.{i}.attention.self.query.bias']= \
            f'encoder.layer.{i}.attention.self_attention.query.bias'
        weight_map[f'encoder.layer.{i}.attention.self.key.weight']= \
            f'encoder.layer.{i}.attention.self_attention.key.weight'
        weight_map[f'encoder.layer.{i}.attention.self.key.bias']= \
            f'encoder.layer.{i}.attention.self_attention.key.bias'
        weight_map[f'encoder.layer.{i}.attention.self.value.weight']= \
            f'encoder.layer.{i}.attention.self_attention.value.weight'
        weight_map[f'encoder.layer.{i}.attention.self.value.bias']= \
            f'encoder.layer.{i}.attention.self_attention.value.bias'
        weight_map[f'encoder.layer.{i}.attention.self.w2e_query.weight']= \
            f'encoder.layer.{i}.attention.self_attention.w2e_query.weight'
        weight_map[f'encoder.layer.{i}.attention.self.w2e_query.bias']= \
            f'encoder.layer.{i}.attention.self_attention.w2e_query.bias'
        weight_map[f'encoder.layer.{i}.attention.self.e2w_query.weight']= \
            f'encoder.layer.{i}.attention.self_attention.e2w_query.weight'
        weight_map[f'encoder.layer.{i}.attention.self.e2w_query.bias']= \
            f'encoder.layer.{i}.attention.self_attention.e2w_query.bias'
        weight_map[f'encoder.layer.{i}.attention.self.e2e_query.weight']= \
            f'encoder.layer.{i}.attention.self_attention.e2e_query.weight'
        weight_map[f'encoder.layer.{i}.attention.self.e2e_query.bias']= \
            f'encoder.layer.{i}.attention.self_attention.e2e_query.bias'
        weight_map[f'encoder.layer.{i}.attention.output.dense.weight']= \
            f'encoder.layer.{i}.attention.output.dense.weight'
        weight_map[f'encoder.layer.{i}.attention.output.dense.bias'] = \
            f'encoder.layer.{i}.attention.output.dense.bias'
        weight_map[f'encoder.layer.{i}.attention.output.LayerNorm.weight'] = \
            f'encoder.layer.{i}.attention.output.LayerNorm.gamma'
        weight_map[f'encoder.layer.{i}.attention.output.LayerNorm.bias'] = \
            f'encoder.layer.{i}.attention.output.LayerNorm.beta'
        weight_map[f'encoder.layer.{i}.intermediate.dense.weight'] = \
            f'encoder.layer.{i}.intermediate.weight'
        weight_map[f'encoder.layer.{i}.intermediate.dense.bias'] = \
            f'encoder.layer.{i}.intermediate.bias'
        weight_map[f'encoder.layer.{i}.output.dense.weight'] = \
            f'encoder.layer.{i}.output.dense.weight'
        weight_map[f'encoder.layer.{i}.output.dense.bias'] = \
            f'encoder.layer.{i}.output.dense.bias'
        weight_map[f'encoder.layer.{i}.output.LayerNorm.weight'] = \
            f'encoder.layer.{i}.output.layernorm.gamma'
        weight_map[f'encoder.layer.{i}.output.LayerNorm.bias'] = \
            f'encoder.layer.{i}.output.layernorm.beta'
    # add pooler
#     weight_map.update(
#         {
#             'pooled_fc.w_0': 'ernie.ernie.dense.weight',
#             'pooled_fc.b_0': 'ernie.ernie.dense.bias',
#             'cls_out_w': 'ernie.dense_1.weight',
#             'cls_out_b': 'ernie.dense_1.bias'
#         }
#    )
    return weight_map

In [29]:
def extract_and_convert(torch_model, ms_model):
    """extract weights and convert to mindspore"""
    print('=' * 20 + 'extract weights' + '=' * 20)
    state_dict = []
    weight_map = build_params_map(attention_num=config.num_attention_heads)
    
    for weight_name, weight_value in torch_model.items():
        if weight_name not in weight_map.keys():
            continue
       
        state_dict.append({'name': weight_map[weight_name], 'data': Tensor(weight_value.numpy())})
        print(weight_name, '->', weight_map[weight_name], weight_value.shape)
    save_checkpoint(state_dict, os.path.join("./luke-large-qa.ckpt"))
    print('=' * 20 + 'extract weights finished' + '=' * 20)

In [30]:
extract_and_convert(torch_model,ms_model)

====================extract weights====================
encoder.layer.0.attention.self.query.weight -> encoder.layer.0.attention.self_attention.query.weight torch.Size([1024, 1024])
encoder.layer.0.attention.self.query.bias -> encoder.layer.0.attention.self_attention.query.bias torch.Size([1024])
encoder.layer.0.attention.self.key.weight -> encoder.layer.0.attention.self_attention.key.weight torch.Size([1024, 1024])
encoder.layer.0.attention.self.key.bias -> encoder.layer.0.attention.self_attention.key.bias torch.Size([1024])
encoder.layer.0.attention.self.value.weight -> encoder.layer.0.attention.self_attention.value.weight torch.Size([1024, 1024])
encoder.layer.0.attention.self.value.bias -> encoder.layer.0.attention.self_attention.value.bias torch.Size([1024])
encoder.layer.0.attention.self.w2e_query.weight -> encoder.layer.0.attention.self_attention.w2e_query.weight torch.Size([1024, 1024])
encoder.layer.0.attention.self.w2e_query.bias -> encoder.layer.0.attention.self_attention.w2

encoder.layer.8.intermediate.dense.weight -> encoder.layer.8.intermediate.weight torch.Size([4096, 1024])
encoder.layer.8.intermediate.dense.bias -> encoder.layer.8.intermediate.bias torch.Size([4096])
encoder.layer.8.output.dense.weight -> encoder.layer.8.output.dense.weight torch.Size([1024, 4096])
encoder.layer.8.output.dense.bias -> encoder.layer.8.output.dense.bias torch.Size([1024])
encoder.layer.8.output.LayerNorm.weight -> encoder.layer.8.output.layernorm.gamma torch.Size([1024])
encoder.layer.8.output.LayerNorm.bias -> encoder.layer.8.output.layernorm.beta torch.Size([1024])
encoder.layer.9.attention.self.query.weight -> encoder.layer.9.attention.self_attention.query.weight torch.Size([1024, 1024])
encoder.layer.9.attention.self.query.bias -> encoder.layer.9.attention.self_attention.query.bias torch.Size([1024])
encoder.layer.9.attention.self.key.weight -> encoder.layer.9.attention.self_attention.key.weight torch.Size([1024, 1024])
encoder.layer.9.attention.self.key.bias -> en

entity_embeddings.entity_embeddings.weight -> entity_embeddings.entity_embeddings.embedding_table torch.Size([500000, 256])
entity_embeddings.entity_embedding_dense.weight -> entity_embeddings.entity_embedding_dense.weight torch.Size([1024, 256])
entity_embeddings.position_embeddings.weight -> entity_embeddings.position_embeddings.embedding_table torch.Size([514, 1024])
entity_embeddings.token_type_embeddings.weight -> entity_embeddings.token_type_embeddings.embedding_table torch.Size([1, 1024])
entity_embeddings.LayerNorm.weight -> entity_embeddings.layer_norm.gamma torch.Size([1024])
entity_embeddings.LayerNorm.bias -> entity_embeddings.layer_norm.beta torch.Size([1024])
qa_outputs.weight -> qa_outputs.weight torch.Size([2, 1024])
qa_outputs.bias -> qa_outputs.bias torch.Size([2])
